In [1]:
import matplotlib.pyplot as plt

import numpy as np
import networkx as nx

from scipy import stats
import pandas as pd
import math

# Generation of WAN network edgelist with symmetrized weights




In [2]:
# Keys for pd dataframe
nodes_keys = ['node_id','iata_code','airport_name','iso_a3','lat','lon']
links_keys = ['source','target','flights','seats']

# PATH where edgelist is stored
period = '2020-11'
OUT_PATH = "./wan_graphs/G_WAN_{}.edgelist".format(period)

# WAN dataset
WAN = pd.read_csv("./wan/wan_nodes.csv")
WAN_edges = pd.read_csv("./wan/wan_link_{}.csv".format(period))

In [3]:
# get airports
airports = list(set(WAN['iata_code'])) # I refer to the IATA code as lid

# Numbers
num_airports = len(airports)
num_edges = len(list(WAN_edges['source']))

In [4]:
# Generate graph
G_WAN = nx.Graph()

# Nodes
for lid in airports:
    G_WAN.add_node(lid)
    
    # add metadata
    for key in nodes_keys:
        G_WAN.nodes[lid][key] = WAN[WAN['iata_code']==lid][key].values[0]

In [5]:
# Add Edges
for i in range(num_edges):
    
    weight = WAN_edges['seats'][i]
    source = WAN_edges['source'][i]
    dest = WAN_edges['target'][i]
    
    source_lid =  WAN[WAN['node_id'] == source]['iata_code'].values[0]
    dest_lid =  WAN[WAN['node_id'] == dest]['iata_code'].values[0]
    
    if G_WAN.has_edge(source_lid, dest_lid):
        # Symmetrize weights
        new_weight = (weight + G_WAN[source_lid][dest_lid]['weight'])/2.0
        G_WAN.add_edge(dest_lid, source_lid, weight=new_weight)
        
    else:
        G_WAN.add_edge(dest_lid, source_lid, weight = weight) # refer to seats as weight

In [6]:
# Save the graph
if 1:
    # Save the edgelist where nodes are references using LIDs
    nx.write_edgelist(G_WAN, OUT_PATH)